In [ ]:
pip install trl bert-score transformers -qqq

In [ ]:
from trl import PPOTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_auxiliary = AutoModelForCausalLM.from_pretrained(model_auxiliary_name)
tokenizer_auxiliary = AutoTokenizer.from_pretrained(model_auxiliary_name)

ppo_trainer = PPOTrainer(model=model_auxiliary, tokenizer=tokenizer_auxiliary) # ...

for original_prompt, correct_answer in dataset:
    rewrite_prompt = model_auxiliary.generate(original_prompt)

    response = model_responder.generate(rewrite_prompt)

    reward = compute_reward(response, correct_answer)

    ppo_trainer.step([original_prompt], [rewrite_prompt], [reward])

In [ ]:
def compute_reward(original_prompt, reformulated_prompt, response, correct_response):
    reward = 0
    if response == correct_response:
        reward += 1.0

    if len(reformulated_prompt) < len(original_prompt):
        reward += 0.2

    reward += compute_semantic_similarity(original_prompt, reformulated_prompt)

    return reward

In [ ]:
from bert_score import score

def compute_semantic_similarity(original_prompt, reformulated_prompt):
    P, R, F1 = score([reformulated_prompt], [original_prompt], lang="en", verbose=False)

    similarity_score = F1[0].item()

    print(f"Similarity Score (BERTScore F1): {similarity_score:.4f}")

    if similarity_score > 0.85:
        reward = 0.3
    elif similarity_score > 0.7:
        reward = 0.1
    else:
        reward = 0.0

    return reward

In [ ]:
## test compute_reward
a = "If a dozen apples cost $3.60, what is the cost of one apple?"
b = "What is the price for one apple if 12 cost $3.60?"
compute_reward(a, b, 4, 4)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Similarity Score (BERTScore F1): 0.9432


1.5